### What does `tf.train.Saver.save` do?

```{python}
tf.train.Saver.save(self, sess, save_path, global_step=None, latest_filename=None, meta_graph_suffix='meta', write_meta_graph=True, write_state=True)
```

[source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/training/saver.py)

Two required arguements:

1. sess (`tf.Session`): a tensorflow Session object
2. save_path (`str`): the path where to write model checkpoint files, used to generate files below
  - **`(save_path)`-`(global_step)`.meta**
  - **`(save_path)`.data-`(step)`-of-`(total)`**
  - **`(save_path)`-`(global_step)`.index**

`.meta` file --> protobuf file `MetaGraphDef`

`.data-????-of-????` and `.index` files --> protobuff file for restoring variables
  
Basically `save_path` is used to detemine:
1. the path and prefix of saved files
2. updating `checkpoint` file

Once `tf.train.Saver` generate file names according to `save_path` and `global_step`, it moves to export **meta graph**

The source code is as following:

![saver-save](images/saver_save.png)


`meta_graph_filename` is now something like `(save_path)-(global_step).meta`

----

### `tf.train.Saver.export_meta_graph`

```tf.train.Saver.export_meta_graph(self, filename=None, collection_list=None, as_text=False, export_scope=None, clear_devices=False)
```

The source code is as following:

![saver-export-meta](images/saver_export_meta_graph.png)

As you can see, it's just a wrapper method for calling `export_meta_graph` function, which is defined in the same source file.

![export-meta](images/export_meta_graph.png)

Again, a wrapped call to `meta_graph.export_scoped_meta_graph`.

----

The source code of `meta_graph.export_scoped_meta_graph` though is not long but requires famililarity with `ProtocolBuffer`, a serailization tool developed by Google. I'll not explain it in this note but you can check those `.proto` files listed below to see the **definitions** for these protobuff objects.

- [meta_graph.proto](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/protobuf/meta_graph.proto)
- [saver.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/saver.proto)
- [graph.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/graph.proto)
- [node_def.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/node_def.proto)

There are many more of them, good luck!

Conceptually, what `meta_graph.export_scoped_meta_graph` does is that exporting data (saver version, op names, ...etc) which can be used to restore the metagraph, which can be used to restore a computational graph. Like this:

```
export_meta_graph
  --> meta_graph.export_scoped_meta_graph
    --> build graph_def from default graph or given graph
       -->create a meta_graph_def by the graph_def
         --> save the meta_graph_def to disk (xxx.meta)
```

Similary, the method call to `tf.train.import_meta_graph` will reverse above process as follwoing:

```
import_meta_grph
  --> meta_graph.read_meta_graph_file to reconstruct meta_graph_def protobuf object
    --> meta_graph.import_scoped_meta_graph to reconstruct the graph object
      --> return a saver which can be used to restore the session
```

Why returning a saver? What is it for?

Well, we can only reconstruct the graph from a meta graph and the variables in the graph are not yet initialized.



---

### What does tf.Saver.restore do?

Very simple, run the restore operation for a given session to restore the state of the session.

![saver-restore](images/saver_restore.png)